# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break


import sys
from data_wrangling import specific_utils, process_monkey_information, specific_utils, process_monkey_information, time_calib_utils, retrieve_raw_data, general_utils, time_calib_class, time_calib_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, monkey_landing_in_ff
from visualization.matplotlib_tools import plot_behaviors_utils
from planning_analysis.show_planning import nxt_ff_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils


import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math
import seaborn as sns
from matplotlib import rc
from os.path import exists
from numpy import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from importlib import reload
import neo
import gc
import rcca
from statsmodels.multivariate.cancorr import CanCorr

import sys
import os, sys
import numpy as np
import torch
from numpy import pi
from matplotlib import pyplot as plt

import numpy as np
import scipy.linalg as linalg
import scipy.interpolate as interpolate
from scipy.signal import fftconvolve


from scipy.io import loadmat
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import sys

import scipy.sparse as sparse
import matplotlib.pylab as plt
import subprocess

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0410"
t_inst = time_calib_class.TimeCalibration(raw_data_folder_path=raw_data_folder_path)
t_inst.retrieve_or_make_monkey_data()
t_inst.make_adjusted_ff_caught_times_df()
t_inst.separate_ff_caught_times_df()
t_inst.txt_and_smr.head(7)

# Calibrate smr markers

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0410"

In [ ]:
channel_signal_output, marker_list, smr_sampling_rate = retrieve_raw_data.extract_smr_data(raw_data_folder_path)
juice_timestamp = marker_list[0]['values'][marker_list[0]['labels'] == 4]
unadj_smr_markers_start_time = marker_list[0]['values'][marker_list[0]['labels']==1][0]
unadj_smr_markers_end_time = juice_timestamp[-1]

In [ ]:
# marker_df = pd.DataFrame({'time': marker_list[0]['values'], 
#                           'label': marker_list[0]['labels']})

In [ ]:
# marker_df_sub = marker_df[marker_df['label'].isin([1, 4])].copy()

In [ ]:
# smr_t_raw = marker_list[0]['values'][marker_list[0]['labels'] == 4]
# df, _, txt_smr_offset_2 = time_calib_utils.make_txt_and_smr_df(smr_t_raw, t_inst.ff_caught_T_sorted)


In [ ]:
txt_smr_offset_2

In [ ]:
ff_caught_T_sorted, ff_index_sorted, ff_real_position_sorted, ff_believed_position_sorted, ff_life_sorted, ff_flash_end_sorted = retrieve_raw_data.make_or_retrieve_ff_info_from_txt_data(raw_data_folder_path)

In [ ]:
ff_caught_T_sorted

In [ ]:
processed_data_folder_path = raw_data_folder_path.replace('raw_monkey_data', 'processed_data')        
npz_file = os.path.join(os.path.join(processed_data_folder_path, 'ff_basic_info.npz'))
loaded_arrays = np.load(npz_file, allow_pickle = True)
ff_caught_T_sorted = loaded_arrays['ff_caught_T_sorted']

In [ ]:
ff_caught_T_sorted

In [ ]:
# smr_t_raw = marker_list[0]['values'][marker_list[0]['labels'] == 4]
# df, _, txt_smr_offset_2 = time_calib_utils.make_txt_and_smr_df(smr_t_raw, t_inst.ff_caught_T_sorted)
# smr_markers_start_time = smr_markers_start_time + txt_smr_offset_2
# smr_markers_end_time = smr_markers_end_time + txt_smr_offset_2

In [ ]:
smr_markers_start_time

# smr time calibration  (based on monkey_x and y)

### get new signal_df

In [ ]:
signal_df = process_monkey_information.make_signal_df(raw_data_folder_path)

In [ ]:
signal_df.head(4)

### test the calib result

In [ ]:
raw_monkey_information = retrieve_raw_data.get_raw_monkey_information_from_txt_data(raw_data_folder_path)
smr_markers_start_time, smr_markers_end_time = time_calib_utils.find_smr_markers_start_and_end_time(raw_data_folder_path)
monkey_information = retrieve_raw_data._trim_monkey_information(raw_monkey_information, smr_markers_start_time, smr_markers_end_time)

In [ ]:
# Test the result

# duration = [1800, 1801]
for start_time in np.arange(100, 1800, 500):
    duration = [start_time, start_time + 3]
    smr_subset = signal_df.loc[signal_df['time'].between(duration[0], duration[1])]
    txt_subset = monkey_information[(monkey_information['time'] >= duration[0]) & (monkey_information['time'] <= duration[1])]
    # plot monkey x and y
    plt.scatter(smr_subset['time'], smr_subset['MonkeyY'], s=2, alpha=0.5)
    plt.scatter(txt_subset['time'], txt_subset['monkey_y'], s=2, alpha=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Monkey Y')
    # plot monkey x and y
    plt.legend(['smr', 'txt'])
    plt.show()

    plt.scatter(smr_subset['time'], smr_subset['MonkeyX'], s=2, alpha=0.5)
    plt.scatter(txt_subset['time'], txt_subset['monkey_x'], s=2, alpha=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Monkey X')
    plt.legend(['smr', 'txt'])
    plt.show()


### lin reg of time offset based on ceiling_of_min_distance

In [ ]:
txt_smr_t_diff_via_xy_df = time_calib_utils.make_or_retrieve_txt_smr_t_diff_via_xy_df(raw_data_folder_path)

In [ ]:
overall_txt_smr_t_linreg_df = pd.DataFrame()
for ceiling_of_min_distance in range(2, 5):
    fig, ax = plt.subplots(figsize=(6, 4))
    df_sub = time_calib_utils.clean_txt_smr_t_diff_via_xy_df_based_on_min_distance(txt_smr_t_diff_via_xy_df, ceiling_of_min_distance=ceiling_of_min_distance)
    ax, txt_smr_t_linreg_df = time_calib_utils.get_linear_regression(df_sub['time'].values, df_sub['time_offset'].values, ax=ax)
    txt_smr_t_linreg_df['ceiling_of_min_distance'] = ceiling_of_min_distance
    overall_txt_smr_t_linreg_df = pd.concat([overall_txt_smr_t_linreg_df, txt_smr_t_linreg_df])
    #plt.show()
overall_txt_smr_t_linreg_df.set_index('ceiling_of_min_distance', inplace=True)
overall_txt_smr_t_linreg_df

### subplots based on ceiling_of_min_distance

In [ ]:
# Create subplots
ceilings = [0.1, 0.5, 1, 2, 3, 5]
num_rows = math.ceil(len(ceilings) / 2)
num_cols = 2
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 10))
axes = axes.flatten()

# Plot each ceiling in a subplot
for i, ceiling in enumerate(ceilings):
    df_sub0 = time_calib_utils.clean_txt_smr_t_diff_via_xy_df_based_on_min_distance(txt_smr_t_diff_via_xy_df, ceiling_of_min_distance=ceiling)
    df_sub = df_sub0.iloc[np.where(time_calib_utils.find_non_outliers_based_on_IQR(df_sub0['time_offset']))[0]].copy()
    sns.scatterplot(data=df_sub, x='time', y='time_offset', hue='min_distance', s=20, alpha=0.5, palette='viridis', ax=axes[i])
    axes[i].set_title(f'ceiling of min_distance = {ceiling}')

# Adjust layout
plt.tight_layout()
plt.show()

# Compare monkey_speed and speeddummy between txt and smr

In [ ]:
monkey_information = t_inst.monkey_information.copy()
monkey_information['monkey_speeddummy'] = ((monkey_information['speed'] > 0.1) | \
                                        (np.abs(monkey_information['ang_speed']) > 0.0035)).astype(int)
monkey_information['monkey_speeddummy_smr'] = ((monkey_information['speed_smr'] > 0.1) | \
                                        (np.abs(monkey_information['ang_speed_smr']) > 0.0035)).astype(int)
diff_df = monkey_information[['point_index', 'time']].copy()
diff_df['monkey_x'] = monkey_information['monkey_x'] - monkey_information['monkey_x_smr']
diff_df['monkey_y'] = monkey_information['monkey_y'] - monkey_information['monkey_y_smr']
diff_df['speed'] = monkey_information['speed'] - monkey_information['speed_smr']
diff_df['ang_speed'] = monkey_information['ang_speed'] - monkey_information['ang_speed_smr']
diff_df['monkey_speeddummy'] = monkey_information['monkey_speeddummy'] - monkey_information['monkey_speeddummy_smr']

## get statistics

### overall

In [ ]:
diff_df.describe()

In [ ]:
# get every 5 percentile for columns in diff_df, and put them into the same df
percentiles = np.arange(0, 100, 1)
percentiles_df = pd.DataFrame({'percentile': percentiles})
for col in diff_df.columns:
    # get the non-NA values in diff_df[col]
    non_na_values = diff_df[col].dropna()
    percentiles_df[col] = np.percentile(non_na_values, percentiles)

In [ ]:
percentiles_df.head(10)

In [ ]:
percentiles_df.tail(10)

### monkey_speeddummy

In [ ]:
monkey_information['monkey_speeddummy'].mean()

In [ ]:
diff_df['monkey_speeddummy'].mean()

In [ ]:
diff_df['monkey_speeddummy'].abs().mean()

## see specific points

### based on monkey_speeddummy

In [ ]:
diff_df[diff_df['monkey_speeddummy'] != 0].iloc[1000:]

In [ ]:
columns = ['point_index', 'time', 'monkey_speeddummy', 'monkey_speeddummy_smr', 'speed', 'speed_smr', 'ang_speed', 'ang_speed_smr', 'monkey_x', 'monkey_x_smr', 'monkey_y', 'monkey_y_smr']

In [ ]:
point_index = 20641
m_sub = monkey_information.loc[point_index-5: point_index+5, columns].copy()
m_sub['whether_the_points'] = np.where(m_sub['point_index'] == point_index, 'yesssss', 'no')
m_sub.set_index('whether_the_points', inplace=True)
m_sub

### based on monkey_speed

In [ ]:
diff_df[diff_df['speed'] > 10].iloc[100:]

In [ ]:
columns = ['point_index', 'time', 'monkey_x', 'monkey_x_smr', 'monkey_y', 'monkey_y_smr', 'speed', 'speed_smr', 'ang_speed', 'ang_speed_smr', 'monkey_speeddummy', 'monkey_speeddummy_smr']

In [ ]:
point_index = 2370
m_sub = monkey_information.loc[point_index-5: point_index+5, columns].copy()
m_sub['whether_the_points'] = np.where(m_sub['point_index'] == point_index, 'yesssss', 'no')
m_sub.set_index('whether_the_points', inplace=True)
m_sub

# Compare scatter within ff reward boundary

In [ ]:
valid_subset = pd.DataFrame({'cur_ff_index': np.arange(len(t_inst.ff_caught_T_sorted)),
                             'time': t_inst.ff_caught_T_sorted})
                             

## txt

In [ ]:
monkey_landing_in_ff.plot_monkey_landings_in_ff(valid_subset, t_inst.monkey_information, t_inst.ff_real_position_sorted)


## smr

In [ ]:
temp_monkey_information = t_inst.monkey_information.copy()
temp_monkey_information['monkey_x'] = temp_monkey_information['monkey_x_smr']
temp_monkey_information['monkey_y'] = temp_monkey_information['monkey_y_smr']

In [ ]:
monkey_landing_in_ff.plot_monkey_landings_in_ff(valid_subset, temp_monkey_information, t_inst.ff_real_position_sorted)

# Compare closest_stop_to_capture

## stop based on both

monkey_speeddummy is based on both smr and txt (if one of them identifies a point as stop, then it is a stop)

In [ ]:
temp_monkey_information = t_inst.monkey_information.copy()

In [ ]:
closest_stop_to_capture_df = monkey_landing_in_ff.get_closest_stop_time_to_all_capture_time(t_inst.ff_caught_T_sorted, temp_monkey_information, t_inst.ff_real_position_sorted, 
                                                                                        cur_ff_index_array=np.arange(len(t_inst.ff_caught_T_sorted)))
closest_stop_to_capture_df.head(3)

In [ ]:
closest_stop_to_capture_df['diff_from_caught_time'].describe()

In [ ]:
# Notice the unit of y axis

plt.figure(figsize=(7, 4))
num_points = closest_stop_to_capture_df.shape[0]
plt.scatter(range(num_points), closest_stop_to_capture_df['diff_from_caught_time'], s=3, color='red')
plt.plot(range(num_points), np.zeros(num_points), color='blue')
plt.title(f'txt: {raw_data_folder_path}')
plt.xlabel('ff index')
plt.ylabel('diff_from_caught_time')
plt.show()

## stop based on txt alone

In [ ]:
temp_monkey_information = t_inst.monkey_information.copy()
temp_monkey_information['monkey_speeddummy'] = ((temp_monkey_information['speed_smr'] > 0.1) | \
                                        (np.abs(temp_monkey_information['ang_speed_smr']) > 0.0035)).astype(int)

In [ ]:
closest_stop_to_capture_df = monkey_landing_in_ff.get_closest_stop_time_to_all_capture_time(t_inst.ff_caught_T_sorted, temp_monkey_information, t_inst.ff_real_position_sorted, 
                                                                                        cur_ff_index_array=np.arange(len(t_inst.ff_caught_T_sorted)))
closest_stop_to_capture_df.head(3)

In [ ]:
closest_stop_to_capture_df['diff_from_caught_time'].describe()

In [ ]:
# Notice the unit of y axis

plt.figure(figsize=(7, 4))
num_points = closest_stop_to_capture_df.shape[0]
plt.scatter(range(num_points), closest_stop_to_capture_df['diff_from_caught_time'], s=3, color='red')
plt.plot(range(num_points), np.zeros(num_points), color='blue')
plt.title(f'txt: {raw_data_folder_path}')
plt.xlabel('ff index')
plt.ylabel('diff_from_caught_time')
plt.show()

## stop based on smr alone

In [ ]:
temp_monkey_information = t_inst.monkey_information.copy()
temp_monkey_information['monkey_speeddummy'] = ((temp_monkey_information['speed_smr'] > 0.1) | \
                                        (np.abs(temp_monkey_information['ang_speed_smr']) > 0.0035)).astype(int)

In [ ]:
closest_stop_to_capture_df2 = monkey_landing_in_ff.get_closest_stop_time_to_all_capture_time(t_inst.ff_caught_T_sorted, temp_monkey_information, t_inst.ff_real_position_sorted, 
                                                                                        cur_ff_index_array=np.arange(len(t_inst.ff_caught_T_sorted)))
closest_stop_to_capture_df2.head(3)

In [ ]:
closest_stop_to_capture_df2['diff_from_caught_time'].describe()

In [ ]:
plt.figure(figsize=(7, 4))
num_points = closest_stop_to_capture_df2.shape[0]
plt.scatter(range(num_points), closest_stop_to_capture_df2['diff_from_caught_time'], s=3, color='red')
plt.plot(range(num_points), np.zeros(num_points), color='blue')
plt.title(f'smr: {raw_data_folder_path}')
plt.xlabel('ff index')
plt.ylabel('smr: diff_from_caught_time')
plt.show()

## all info based on smr

In [ ]:
raw_signal_df = process_monkey_information.get_raw_signal_df(raw_data_folder_path)
monkey_information = retrieve_raw_data.get_trimmed_monkey_information(raw_data_folder_path)
monkey_information = process_monkey_information._add_smr_file_info_to_monkey_information(monkey_information, raw_signal_df)
monkey_information.rename(columns={'MonkeyX': 'monkey_x_smr', 'MonkeyY': 'monkey_y_smr', 'AngularV': 'ang_speed_smr'}, inplace=True)
monkey_information = process_monkey_information.get_monkey_speed_and_dw_from_smr_info(monkey_information)


In [ ]:
temp_monkey_information = monkey_information.copy()
temp_monkey_information['monkey_speeddummy'] = ((temp_monkey_information['speed_smr'] > 0.1) | \
                                        (np.abs(temp_monkey_information['ang_speed_smr']) > 0.0035)).astype(int)

In [ ]:
closest_stop_to_capture_df2 = monkey_landing_in_ff.get_closest_stop_time_to_all_capture_time(t_inst.ff_caught_times_df['smr_t_raw'], temp_monkey_information, t_inst.ff_real_position_sorted, 
                                                                                        cur_ff_index_array=np.arange(len(t_inst.ff_caught_times_df)))
closest_stop_to_capture_df2.head(3)

In [ ]:
closest_stop_to_capture_df2['diff_from_caught_time'].describe()

In [ ]:
plt.figure(figsize=(7, 4))
num_points = closest_stop_to_capture_df2.shape[0]
plt.scatter(range(num_points), closest_stop_to_capture_df2['diff_from_caught_time'], s=3, color='red')
plt.plot(range(num_points), np.zeros(num_points), color='blue')
plt.title(f'smr: {raw_data_folder_path}')
plt.xlabel('ff index')
plt.ylabel('smr: diff_from_caught_time')
plt.show()

# Compare smr mean vs median

In [ ]:
signal_df = process_monkey_information.make_signal_df(raw_data_folder_path)
monkey_information = t_inst.monkey_information.copy()
time_bins = general_utils.find_time_bins_for_an_array(monkey_information['time'].values)

# add time_box to monkey_information
monkey_information.loc[:, 'time_box'] = np.arange(1, len(monkey_information)+1)
signal_df.loc[:, 'time_box'] = np.digitize(signal_df['time'].values, time_bins)

# use groupby and then find average for LDy, LDz, RDy, RDz
condensed_signal_df = signal_df[['time_box', 'LDy', 'LDz', 'RDy', 'RDz', 'MonkeyX', 'MonkeyY', 'LateralV', 'ForwardV', 'AngularV']]
condensed_signal_df['radius'] = np.sqrt(condensed_signal_df['MonkeyX']**2 + condensed_signal_df['MonkeyY']**2)
mean_df = condensed_signal_df.groupby('time_box').mean().reset_index(drop=False)
median_df = condensed_signal_df.groupby('time_box').median().reset_index(drop=False)
mean_df.index = mean_df.time_box.values
median_df.index = median_df.time_box.values
diff_df = (mean_df - median_df)

In [ ]:
condensed_signal_df.groupby('time_box').count().reset_index(drop=False)

## get statistics

In [ ]:
diff_df.describe()

## see specific points

In [ ]:
diff_df[diff_df['AngularV'] > 10]

In [ ]:
window = 5 

In [ ]:
box = 178818
median_df[median_df['time_box'].between(box-window, box+window)]

In [ ]:
mean_df[mean_df['time_box'].between(box-window, box+window)]

In [ ]:
condensed_signal_df[condensed_signal_df['time_box'].between(box-1, box+1)]

# Compare capture time between data

## compare capture time

### txt vs smr

In [ ]:
t_inst.compare_txt_and_smr_with_boxplot()

In [ ]:
t_inst.compare_txt_and_smr_with_scatterplot()

### txt vs neural

In [ ]:
t_inst.compare_txt_and_neural_with_scatterplot()

### neural vs smr

In [ ]:
t_inst.compare_neural_and_smr_with_scatterplot()

# Neural data calibration

## verify that the calibration is correct

In [ ]:
calibrated_neural_event_t = time_calib_utils.calibrate_neural_data_time(t_inst.ff_caught_times_df['neural_t_raw_ext'].values, t_inst.raw_data_folder_path, t_inst.ff_caught_T_sorted, show_plot=True)

In [ ]:
temp_array = calibrated_neural_event_t - t_inst.ff_caught_T_sorted[:len(calibrated_neural_event_t)]
# make a boxplot on temp_array
plt.figure(figsize=(7, 4))
plt.boxplot(temp_array)
plt.title('Difference between calibrated adjusted neural event time and ff caught time')
plt.show()

In [ ]:
temp_df = pd.DataFrame({'diff': temp_array})
temp_df.describe()

# Plot ff caught time

## Bruno

In [ ]:
additional_plots.plot_ff_caught_time(monkey_name='monkey_Bruno')

## Schro

In [ ]:
additional_plots.plot_ff_caught_time(monkey_name='monkey_Schro')